# MATCH_RECOGNIZE Intro

MATCH_RECOGNIZE is used to identify sequences of rows that satisfy a pattern of conditions. For example, you could:
- Identify sales trends by getting sequences of small customer orders follwed by a large order.
- Collect customer action sequences between log-in and final purchase.

Syntax:
```
SELECT * FROM input_table
MATCH_RECOGNIZE (
    [ PARTITION BY partition_expr ]
    ORDER BY order_expr
    MEASURES measures_expr AS measures_column
    [ ONE ROW PER MATCH ]
    [ AFTER MATCH SKIP { PAST LAST ROW | TO NEXT ROW } ]
    PATTERN ( pattern_expr )
    DEFINE symbol AS symbol_expr[, …]
)
```





# Create input table

In [ ]:
%%bigquery --location=us-east7
CREATE SCHEMA IF NOT EXISTS m_r_demo OPTIONS(location='us-east7');
CREATE OR REPLACE TABLE m_r_demo.Orders(id INT64, country STRING, price INT64) AS
SELECT * FROM UNNEST([
  STRUCT(1, "USA", 200),
  STRUCT(2, "USA", 300),
  STRUCT(3, "France", 50),
  STRUCT(4, "USA", 60),
  STRUCT(5, "Mexico", 70),
  STRUCT(6, "USA", 24),
  STRUCT(7, "China", 60),
  STRUCT(8, "USA", 55),
  STRUCT(9, "China", 40),
  STRUCT(10, "USA", 38),
  STRUCT(11, "USA", 150),
  STRUCT(12, "France", 180)
]);

Query is running:   0%|          |

""


In [ ]:
%%bigquery --location=us-east7
SELECT * FROM m_r_demo.Orders
ORDER BY (country="USA"), id

Query is running:   0%|          |

Downloading:   0%|          |

,id,country,price
0,3,France,50
1,5,Mexico,70
2,7,China,60
3,9,China,40
4,12,France,180
5,1,USA,200
6,2,USA,300
7,4,USA,60
8,6,USA,24
9,8,USA,55


In the `Orders` table, we will identify patterns of small orders followed by a single large order.

# Query 1: Non-overlapping match (Default behavior)

First, we search for non-overlapping patterns of large orders preceded by small orders.

In [ ]:
%%bigquery --location=us-east7
SELECT *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    ARRAY_AGG(price) AS orders
  PATTERN (small* large)
  DEFINE
    small AS price < 100,
    large AS price >= 100
);

Query is running:   0%|          |

Downloading:   0%|          |

,location,orders
0,USA,[200]
1,USA,[300]
2,USA,"[60, 24, 55, 38, 150]"
3,non-USA,"[50, 70, 60, 40, 180]"


# Query 2: Overlapping match

Now, we include overlapping matches in our search, so that every small order leading up to a large order starts its own match.

In [ ]:
%%bigquery --location=us-east7
SELECT
  *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    ARRAY_AGG(price) AS orders
  AFTER MATCH SKIP TO NEXT ROW
  PATTERN (small* large)
  DEFINE
    small AS price < 100,
    large AS price >= 100
);


Query is running:   0%|          |

Downloading:   0%|          |

,location,orders
0,USA,[200]
1,USA,[300]
2,USA,"[60, 24, 55, 38, 150]"
3,USA,"[24, 55, 38, 150]"
4,USA,"[55, 38, 150]"
5,USA,"[38, 150]"
6,USA,[150]
7,non-USA,"[50, 70, 60, 40, 180]"
8,non-USA,"[70, 60, 40, 180]"
9,non-USA,"[60, 40, 180]"


# Query 3: longest-match mode off (Default behavior)

By default, pattern searching will prioritize earlier matches and respect the regex-style pattern rules.

In [ ]:
%%bigquery --location=us-east7
SELECT
  *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    ARRAY_AGG(price ORDER BY price) AS orders
  PATTERN (small+?)  -- Reluctantly find small consecutive orders
  DEFINE
    small AS price < 100
);

Query is running:   0%|          |

Downloading:   0%|          |

,location,orders
0,USA,[60]
1,USA,[24]
2,USA,[55]
3,USA,[38]
4,non-USA,[50]
5,non-USA,[70]
6,non-USA,[60]
7,non-USA,[40]


# Query 4: longest-match mode on

Sometimes, we want to prioritize the longest matches possible, using use_longest_match = TRUE

In [ ]:
%%bigquery --location=us-east7
SELECT
  *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    ARRAY_AGG(price) AS orders
  PATTERN (small+?)  -- Reluctantly find small consecutive orders, with a preference for longer matches
  DEFINE
    small AS price < 100
  OPTIONS (use_longest_match = TRUE)
);

Query is running:   0%|          |

Downloading:   0%|          |

,location,orders
0,USA,"[60, 24, 55, 38]"
1,non-USA,"[50, 70, 60, 40]"


# Query 5: Special functions in MEASURES

* MATCH_NUMBER() returns the 1-based index of each match
* MATCH_ROW_NUMBER() returns the 1-based index of each row in a match
* CLASSIFIER() returns the symbol name that each row in a match matched to
* FIRST() returns the first row in a match
* LAST() returns the last row in a match

In [ ]:
%%bigquery --location=us-east7
SELECT
  *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    MATCH_NUMBER() AS match_num,
    ARRAY_AGG(MATCH_ROW_NUMBER()) AS match_row_nums,
    ARRAY_AGG(CLASSIFIER()) AS matched_syms,
    FIRST(STRUCT(id, price)) AS first_order,
    LAST(STRUCT(id, price)) AS last_order
  PATTERN (small* large)
  DEFINE
    small AS price < 100,
    large AS price >= 100
);

Query is running:   0%|          |

Downloading:   0%|          |

,location,match_num,match_row_nums,matched_syms,first_order,last_order
0,USA,1,[1],[large],"{'id': 1, 'price': 200}","{'id': 1, 'price': 200}"
1,USA,2,[1],[large],"{'id': 2, 'price': 300}","{'id': 2, 'price': 300}"
2,USA,3,"[1, 2, 3, 4, 5]","[small, small, small, small, large]","{'id': 4, 'price': 60}","{'id': 11, 'price': 150}"
3,non-USA,1,"[1, 2, 3, 4, 5]","[small, small, small, small, large]","{'id': 3, 'price': 50}","{'id': 12, 'price': 180}"


# Query 6: Symbol qualification in MEASURES

To only aggregate rows that matched a certin symbol, qualify the column references in MEASURES with the symbol name.

In [ ]:
%%bigquery --location=us-east7
SELECT
  *
FROM (
  SELECT
    IF(country = 'USA',
      'USA',
      'non-USA'
    ) AS location,
    *
  FROM m_r_demo.Orders
) MATCH_RECOGNIZE(
  PARTITION BY location
  ORDER BY id
  MEASURES
    ARRAY_AGG(large.price) AS large_orders,
    LAST(STRUCT(small.id, small.price, MATCH_ROW_NUMBER() AS row_num)) AS last_small_order
  PATTERN (small* large)
  DEFINE
    small AS price < 100,
    large AS price >= 100
);

Query is running:   0%|          |

Downloading:   0%|          |

,location,large_orders,last_small_order
0,USA,[200],None
1,USA,[300],None
2,USA,[150],"{'id': 10, 'price': 38, 'row_num': 4}"
3,non-USA,[180],"{'id': 9, 'price': 40, 'row_num': 4}"
